In [1]:
import random # library for random number generation
import numpy as np # library for vectorized computation
import pandas as pd # library to process data as dataframes

import matplotlib.pyplot as plt # plotting library
# backend for rendering plots within the browser
%matplotlib inline 

from sklearn.cluster import KMeans 
from sklearn.datasets.samples_generator import make_blobs

In [2]:
pip install beautifulsoup4

     |████████████████████████████████| 112kB 6.4MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install lxml

     |████████████████████████████████| 5.8MB 21.2MB/s eta 0:00:01     |██████████████████████          | 4.0MB 21.2MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
from bs4 import BeautifulSoup
import requests

In [5]:
df = pd.read_csv("vancouver coordinates 2.csv") 

In [6]:
df.head()

,V3A,Langley Township,(Langley City)-Langley Township,49.100002,-122.657128
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993


In [7]:
df.columns=["Postal Code", "City", "Neighbourhood", "Latitude","Longitude"]

In [8]:
df.head()

,Postal Code,City,Neighbourhood,Latitude,Longitude
0,V4A,Surrey,Southwest-Surrey,49.044655,-122.869163
1,V5A,Burnaby,(Government Road / Lake City / SFU / Burnaby M...,49.276301,-122.946971
2,V6A,Vancouver,(Strathcona / Chinatown / Downtown Eastside)- ...,49.279935,-123.090704
3,V7A,Richmond,South- Richmond,49.134223,-123.099148
4,V3B,Port Coquitlam,Central- Port Coquitlam,49.259167,-122.746993


In [9]:
df.shape

(89, 5)

In [10]:
df.tail()

,Postal Code,City,Neighbourhood,Latitude,Longitude
84,V7Y,Vancouver,(Pacific Centre)-Vancouver,49.282728,-123.118463
85,V2Z,Langley Township,Southwest-Langley Township,49.065755,-122.582949
86,V3Z,Surrey,Lower East-Surrey,49.048837,-122.693318
87,V5Z,Vancouver,(East Fairview / South Cambie)-Vancouver,49.233483,-123.120701
88,V6Z,Vancouver,(SW Downtown)-Vancouver,49.275944,-123.131166


In [11]:
CLIENT_ID = 'PNBZRK0AATNMVG3M1CVPRL2IYCPPD5VI52ROWJKW53OZDUXM' # Foursquare ID
CLIENT_SECRET = 'IDM0XD24USYGN05IGYR5VI35ELC1UCWUA11JM3MF3AUDLVR3' #  Foursquare Secret
VERSION = '20180604'

In [12]:
def getNearbyVenues(names, latitudes, longitudes):
    radius=500
    LIMIT=100
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
vancouver_venues = getNearbyVenues(names=df['Neighbourhood'],
                                   latitudes=df['Latitude'],
                                   longitudes=df['Longitude']
                                  )

Southwest-Surrey
(Government Road / Lake City / SFU / Burnaby Mountain)-Burnaby
(Strathcona / Chinatown / Downtown Eastside)- Vancouver


In [ ]:
vancouver_venues.head()

In [ ]:
vancouver_venues.groupby('Neighbourhood').count()

In [ ]:
# one hot encoding
vancouver_onehot = pd.get_dummies(vancouver_venues[['Venue Category']], prefix="", prefix_sep="")

In [ ]:
vancouver_onehot['Neighbourhood'] = vancouver_venues['Neighbourhood'] 

In [ ]:
fixed_columns = [vancouver_onehot.columns[-1]] + list(vancouver_onehot.columns[:-1])
vancouver_onehot = vancouver_onehot[fixed_columns]

vancouver_onehot.head()

In [ ]:
vancouver_onehot.shape

In [ ]:
vancouver_grouped = vancouver_onehot.groupby('Neighbourhood').mean().reset_index()
vancouver_grouped

In [ ]:
vancouver_grouped.shape

In [ ]:
num_top_venues = 5

for hood in vancouver_grouped['Neighbourhood']:
    print("----"+hood+"----")
    temp = vancouver_grouped[vancouver_grouped['Neighbourhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')